# Car ride-share potential in mid-size U.S. cities from geographic spread

## (notebook 2)

Second notebook for the IBM Data Science Specialization on Coursera, consolidating and cleaning up the first notebook, to continue with less clutter.

## Copy-forward: All imports so far

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re
import math
import json

!conda install -c conda-forge geopy=1.18.1 --yes


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.91 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.57 MB/s


In [2]:
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
!conda install -c conda-forge folium=0.8.0 --yes
import folium


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.8.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  53.59 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  26.70 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.27 MB/s
folium-0.8.0-p 100% |################################| Time: 0:00:00  46.00 MB/s


In [31]:
import time


SyntaxError: invalid syntax (<ipython-input-31-d1bc98edac96>, line 2)

### Some new imports for this notebook

(added up top to keep the below a bit cleaner)

In [43]:
!conda install -c conda-forge pandas-profiling=1.4.1 --yes
import pandas_profiling

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
pandas-profiling          1.4.1                      py_1    conda-forge


## Copy-forward: Mid-size U.S. cities

In [4]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_United_States_cities_by_population&oldid=883568308'
website_url = requests.get(url).text
soup = BeautifulSoup(website_url, 'lxml')
city_table = soup.find('table', { 'class' : 'wikitable sortable' })
print("{}\n\n   [...]\n\n{}".format(str(city_table)[:500].replace('\n', '').replace('<tr>', '\n\n<tr>'), str(city_table)[-500:]))


<table class="wikitable sortable" style="text-align:center"><tbody>

<tr><th>2017<br/>rank</th><th>City</th><th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[5]</a></sup></th><th>2017<br/>estimate</th><th>2010<br/>Census</th><th>Change</th><th colspan="2">2016 land area</th><th colspan="2">2016 population density</th><th>Location</th></tr>

<tr><td>1</td><td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York 

   [...]

"latitude">38°21′14″N</span> <span class="longitude">121°58′22″W</span></span></span><span class="geo-multi-punct">﻿ / ﻿</span><span class="geo-default"><span class="vcard"><span class="geo-dec" title="Maps, aerial photos, and other data for this location">38.3539°N 121.9728°W</span><span style="display:none">﻿ / <span class="geo">38.3539; -121.9728</span></span><span style="display:none">﻿ (<span class="fn org">Vacaville</span>)</span></span></span></a></span></small>
</td></tr></tbody></table>


In [5]:
l = []

table_rows = city_table.find_all('tr')
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td]
    if len(row) < 1:
        print("(ignoring empty row)")
        test_size = 0
    else:
        test_size = int(row[3].replace(',', ''))
        
    if test_size >= 300000 and test_size <= 400000:
        city_name = re.sub('\[.*\]', '', row[1])
        city_state = row[2]
        city_estd_pop2017 = test_size
        city_latlongraw = re.sub('^.*/', '', re.sub('\(.*\)', '', row[10])).replace(' ', '')
        # strip non-ASCII residue
        city_latlongraw = city_latlongraw.encode('ascii',errors='ignore').decode()
        city_lat = float(re.sub(';.*$', '', city_latlongraw))
        city_long = float(re.sub('^.*;', '', city_latlongraw))
        l.append([city_name, city_state, city_estd_pop2017, city_lat, city_long])

cities_df = pd.DataFrame(l)
cities_df.columns = ['City name', 'City state', 'Population', 'Latitude', 'Longitude']
print(cities_df)


(ignoring empty row)
         City name    City state  Population  Latitude  Longitude
0        Arlington         Texas      396394   32.7007   -97.1247
1      New Orleans     Louisiana      393292   30.0534   -89.9345
2          Wichita        Kansas      390591   37.6907   -97.3459
3        Cleveland          Ohio      385525   41.4785   -81.6794
4            Tampa       Florida      385430   27.9701   -82.4797
5      Bakersfield    California      380874   35.3212  -119.0183
6           Aurora      Colorado      366623   39.6880  -104.6897
7          Anaheim    California      352497   33.8555  -117.7601
8         Honolulu        Hawaii      350395   21.3243  -157.8476
9        Santa Ana    California      334136   33.7363  -117.8830
10       Riverside    California      327728   33.9381  -117.3932
11  Corpus Christi         Texas      325605   27.7543   -97.1734
12       Lexington      Kentucky      321959   38.0407   -84.4583
13        Stockton    California      310496   37.9763 

## Copy-forward: Foursquare functions

In [6]:
# (paste secret from local file)


In [7]:
print('CLIENT_ID set: {}'.format(CLIENT_ID is not None))
print('CLIENT_SECRET set: {}'.format(CLIENT_SECRET is not None))

VERSION = '20180605' # Foursquare API version


CLIENT_ID set: True
CLIENT_SECRET set: True


### >>> getVenuesNearLatLong

In [8]:
def getVenuesNearLatLong(latitude, longitude, radius=500, limit=100, verbose=True):
    
    venues_list=[]
                
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            latitude, 
            longitude, 
            radius, 
            limit)
            
    # make the GET request (on error try four more times before giving up)
    num_tries = 0
    results = [] # assume no venues if persistent error
    
    while num_tries < 5:
        num_tries +=1
        try:
            results_raw = requests.get(url)
            results = results_raw.json()["response"]['groups'][0]['items']
        except:
            print('(err)', end='')
            time.sleep(2) # sleep for two seconds, then retry
        
    # return only relevant information for each nearby venue
    venues_list.append([(
            latitude, 
            longitude, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    if (len(results) > 0):
        nearby_venues.columns = [
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Category']
    
    if verbose:
        print('found {} venues within {} meters of {}/{}'.format(len(results), radius, latitude, longitude))
    else:
        print('{}'.format(len(results)), end='. ')
    
    return(nearby_venues)


### >>> get_venues_in_hex_grid

In [9]:
def get_venues_in_hex_grid(latitude, longitude, venues_dict, this_coord, radius=500, limit=100, new_coords=[], verbose=True):
    '''
    Calls Foursquare in a hex grid around a given coordinate point. If venues
    have already been searched on one of the hex grid points, that result is
    kept and no new search is executed.
    
    Parameters:
    
    latitude and longitude are as of the origin coordinate (0, 0),
    venues_dict are the venues found so far (dictionary keys are a coordinate tuple),
    this_coord is the center coordinate around which the hex grid is to be searched,
    radius is the radius [meters] to search around a coordinate point,
    limit is the maximum number of venues to return from a Foursquare search.
    new_coords is a list of coordinate points that wasn't probed yet
    
    Returns a list of new coordinate tuples appended to the new_coords parameter, if any
    '''
    
    r_earth = 6378000. # approximate radius of the Earth in meters
    pi = math.pi
    sqrt_three = math.sqrt(3.)
    overlap = 1.4 # 40% overlap
    
    cx = this_coord[0] # center X
    cy = this_coord[1] # center Y
    hex_coords = [ (cx-1,cy), (cx+1, cy), (cx,cy-1), (cx,cy+1), (cx-1,cy+1), (cx+1,cy-1) ] # the gex grid around this_coord
    
    if (cx, cy) in new_coords:
        new_coords.remove((cx, cy))
    
    for this_hex in hex_coords:
        if not this_hex in venues_dict:
            # the coordinate has not been searched for
            
            # get the x- and y-step from a hex grid; start with a square grid (letting the circles overlap a bit):
            dx_square = this_hex[0] * radius * ( overlap / 2. )
            dy_square = this_hex[1] * radius * ( overlap / 2. )
            # now convert to a hex grid:
            dx = dx_square + dy_square / 2.
            dy = dy_square * ( sqrt_three / 2. )
            # approximate the center point's latitude and longitude assuming locally flat Earth
            hex_latitude  = latitude  + (dy / r_earth) * (180 / pi);
            hex_longitude = longitude + (dx / r_earth) * (180 / pi) / math.cos(latitude * pi/180);
            
            if verbose:
                print('getting coordinate {}...'.format(this_hex))
            else:
                print('({},{}):'.format(this_hex[0], this_hex[1]), end='')
                
            this_venues = getVenuesNearLatLong(hex_latitude, hex_longitude, radius=radius, limit=limit, verbose=verbose)
            venues_dict[this_hex] = this_venues
            if not this_hex in new_coords:
                new_coords.append(this_hex)
    
    return new_coords


### >>> make_map_from_dict

After upgrading to folium 0.8.0 the maps didn't behave well at times for large data sets. Therefore I added some simplification and a failsafe abort. It only affects the looks, but not the statistics.

In [209]:
def make_map_from_dict(lat_orig, long_orig, venues_dict, zoom_start,
                       city_name='(city_name)', city_state='(city_state)',
                       radius_zoom=1.0, width=600, height=600,
                       no_touch=True,
                       zoom_control=False,
                       failsafe_abort_num=4000,
                       simple_rendering=False):
    
    if zoom_control == False:
        min_zoom = zoom_start
        max_zoom = zoom_start
    else:
        min_zoom = 0
        max_zoom = 16
    
    new_map = folium.Map(
        location=[lat_orig, long_orig],
        zoom_start=zoom_start,
        width=width,
        height=height,
        control_scale=True,
        no_touch=no_touch,
        min_zoom=min_zoom,
        max_zoom=max_zoom
    )
    
    num_markers=0

    # add markers to map
    for coords, venues in venues_dict.items():
        num_markers += 1
        if (venues.shape[0] > 0) and (num_markers < failsafe_abort_num): 
            if simple_rendering:
                folium.Circle(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    color=None,
                    fill=True,
                    fill_color='#005090',
                    fill_opacity=0.7).add_to(new_map)
            else:
                label = '{}, # venues: {}'.format(coords, venues.shape[0])
                label = folium.Popup(label, parse_html=True)
                folium.CircleMarker(
                    [venues['Latitude'][0], venues['Longitude'][0]],
                    radius=venues.shape[0] * radius_zoom,
                    popup=label,
                    color='blue',
                    fill=True,
                    fill_color='#3186cc',
                    fill_opacity=0.7,
                    parse_html=False).add_to(new_map)
        
    legend_html = ('<div style="position: fixed; top: 30px; left: 50px; width: 450px;' 
                + 'height: 30px; border: 2px solid grey; z-index: 9999; font-size: 16px; background-color: white">' 
                + '&nbsp;{},&nbsp;{}' 
                + '</div>').format(
                     city_name,
                     city_state
                     )
    new_map.get_root().html.add_child(folium.Element(legend_html))
     
    return new_map


### >>> find_venues_geo_distribution

In [11]:
def find_venues_geo_distribution(cities_df, city_index, max_coords_tested=100, radius=1500, limit=100, verbose=True):
    
    city_name = cities_df['City name'][city_index]
    city_state = cities_df['City state'][city_index]
    
    # initialize venues_dict with the venues dataframe at the origin
    venues_dict = {}
    origin_coord = (0,0)
    lat_orig = cities_df.Latitude[city_index]
    long_orig = cities_df.Longitude[city_index]
    print('[test #1 of {}] (0,0):'.format(max_coords_tested), end='')
    venues_df = getVenuesNearLatLong(lat_orig, long_orig, radius=radius, verbose=verbose)
    venues_dict[origin_coord] = venues_df
    
    # mark the origin as the first (and only) coordinate point not yet explored
    new_coords = [(0, 0)]
    num_coords_tested = 1
    
    while num_coords_tested < max_coords_tested:
        
        highest_venues = -1
        new_test_coord = None
        
        for this_coord in new_coords:
            venues_df = venues_dict[this_coord]
            if venues_df.shape[0] > highest_venues:
                new_test_coord = this_coord
                highest_venues = venues_df.shape[0]
        
        # call the hex grid exploration function
        num_coords_tested += 1
        print('[test #{} of {}]'.format(num_coords_tested, max_coords_tested), end=' ')
        new_coords = get_venues_in_hex_grid(lat_orig, long_orig, venues_dict, new_test_coord, radius=radius, new_coords=new_coords, limit=limit, verbose=verbose )
        
    return lat_orig, long_orig, venues_dict, city_name, city_state


## Determine aggregates describing the geographical distribution

In order to cluster cities by shape of their venues, we need to calculate aggregate variables:

* Determine the center of the venues by averaging each coordinate point, weighed by the number of venues there.
* Determine the mean distance of all venues from that center.
* Determine the standard deviation of the distance distribution, as well as skewedness and kurtosis ("peaky-ness"). Once we have these values, all details on the geographic venues distribution will be discarded.

But first we have to simplify the dataframe, to sum up the number of indicator venues as compared to the total number of venues.

### Check whether a venue is of an indicator type

Define a vendor indicator function now so that the remainder of the algorithm can be developed and tested. For the sake of this capstone assignment, I'll simply assume to make a study for a client, and the client gave me that list. It is aimed at indicating a population that is willing to try out new and unusual things, in the context of sharing transportation.

### >>> is_indicator_venue

In [12]:
def is_indicator_venue(venue_type):
    if venue_type is None:
        return False
    
    magic_words = [
        'yoga',
        'salad',
        'coworking',
        'alternative',
        'bike',
        'fitness',
        'running',
        'jogging',
        'cycling',
        'cycle',
        'athletics',
        'gluten',
        'health',
        'recreation',
        'tennis',
        'vegetarian',
        'vegan',
        'tennis',
        'disc golf',
        'pilates',
        'share',
        'sharing',
        'incubat',
        'innovat'
    ]
    return any(substring in venue_type.lower() for substring in magic_words)

print('Perform some tests on indicator venues:')
print(is_indicator_venue(None))
print(is_indicator_venue(''))
print(is_indicator_venue('Salad Bar'))
print(is_indicator_venue('Chinese Restaurant'))
print(is_indicator_venue('Coworking space'))


Perform some tests on indicator venues:
False
False
True
False
True


### Aggregate venues_dict into a DataFrame with one row per coordinate

For all coordinates, count the number of venues at that coordinate as well as the number of indicator venues

### >>> aggregate_venues_dict

In [13]:
def aggregate_venues_dict(venues_dict):
    venues_agg = []
    venues_types = []
    for coord, venues in venues_dict.items():
        num_venues = venues.shape[0]
        num_indicators = 0
        if num_venues > 0:
            for this_type in venues['Venue Category'].values:
                venues_types.append(this_type)
                if is_indicator_venue(this_type):
                    num_indicators += 1
        venues_agg.append([coord, num_venues, num_indicators])
    venues_agg_df = pd.DataFrame(venues_agg)
    venues_agg_df.columns = ['coord', 'num_venues', 'num_indicators']
    return venues_agg_df, set(venues_types)


Do a quick test: Get 10 coordinates (plus a few surrounding ones) from the third city in the list (index 2):

In [14]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=2, radius=800, max_coords_tested=10, verbose=False)


[test #1 of 10] (0,0):43. [test #2 of 10] (-1,0):33. (1,0):46. (0,-1):58. (0,1):18. (-1,1):16. (1,-1):58. [test #3 of 10] (-1,-1):44. (0,-2):43. (1,-2):53. [test #4 of 10] (2,-1):78. (2,-2):55. [test #5 of 10] (3,-1):87. (2,0):83. (3,-2):73. [test #6 of 10] (4,-1):74. (3,0):79. (4,-2):52. [test #7 of 10] (2,1):36. (1,1):28. [test #8 of 10] (4,0):55. (3,1):29. [test #9 of 10] (5,-1):24. (5,-2):24. [test #10 of 10] (3,-3):29. (4,-3):16. 

In [15]:
print("{}, {}".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
venues_agg_df.head(10)


Wichita, Kansas


,coord,num_venues,num_indicators
0,"(5, -1)",24,1
1,"(5, -2)",24,1
2,"(0, 0)",43,4
3,"(-1, 0)",33,1
4,"(3, -3)",29,0
5,"(3, 0)",79,2
6,"(2, -1)",78,1
7,"(2, 1)",36,1
8,"(3, -2)",73,0
9,"(2, -2)",55,0


The `aggregate_venues_dict` function also returns a set of all venues, let's loop through all of them and test the indicator function.

In [16]:
for this_type in venues_types:
    print('{}:  {}'.format(is_indicator_venue(this_type), this_type))
    

False:  Coffee Shop
False:  Gay Bar
False:  Butcher
False:  Liquor Store
False:  Arts & Crafts Store
True:  Vegetarian / Vegan Restaurant
False:  Baseball Stadium
False:  Convenience Store
False:  Gourmet Shop
False:  Medical Center
False:  Japanese Restaurant
False:  Taco Place
False:  Sandwich Place
False:  Italian Restaurant
False:  Brewery
False:  Construction & Landscaping
False:  Dance Studio
False:  Cafeteria
False:  Intersection
False:  Salon / Barbershop
False:  Tourist Information Center
False:  Sporting Goods Shop
False:  Design Studio
False:  Business Service
False:  Hookah Bar
False:  Music Store
True:  Tennis Court
False:  New American Restaurant
False:  Electronics Store
False:  Bakery
False:  Vietnamese Restaurant
False:  Other Repair Shop
False:  Science Museum
False:  Pizza Place
False:  Monument / Landmark
False:  Antique Shop
False:  Latin American Restaurant
False:  Grocery Store
False:  Tea Room
False:  Hot Dog Joint
False:  Concert Hall
False:  Diner
False:  Rent

Works.

### Find coordinate center of all venues

In order to analyze the geographic spread of the venues, the center of the venues has to be determined first. This is done by averaging all coordinates, weighted by the number of venues there.

### >>> find_venues_distance_to_center

In [17]:
def find_venues_distance_to_center(venues_agg_df):
    '''
    Appends or updates x/y and distance-to-center columns of venues_agg_df.
    Also returns some aggregates that were calculated along the way.
    '''
    
    x, y = zip(*venues_agg_df['coord'])
    total_num_venues = venues_agg_df['num_venues'].sum()
    total_num_indicators = venues_agg_df['num_indicators'].sum()
    center_x = ( x * venues_agg_df['num_venues'] ).sum() / total_num_venues
    center_y = ( y * venues_agg_df['num_venues'] ).sum() / total_num_venues
    dx = x - center_x
    dy = y - center_y
    dist2 = dx * dx + dy * dy
    venues_agg_df['dist'] = np.sqrt(dist2)
    
    return center_x, center_y, total_num_venues, total_num_indicators


In [18]:
# test the function
center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
print('Test set coordinate center of {0:}, {1:} is at ({2:.2f}, {3:.2f}). Indicator / total venues: {4:} / {5:} ({6:.1f}%)'.format(
    city_name, city_state, center_x, center_y, total_num_indicators, total_num_venues,
    total_num_indicators * 100 / total_num_venues))
print('\nFirst five Euclidean coordinate distances to the center:\n\n{}'.format(venues_agg_df.head(5)))



Test set coordinate center of Wichita, Kansas is at (2.04, -0.84). Indicator / total venues: 24 / 1234 (1.9%)

First five Euclidean coordinate distances to the center:

     coord  num_venues  num_indicators      dist
0  (5, -1)          24               1  2.966477
1  (5, -2)          24               1  3.182609
2   (0, 0)          43               4  2.202692
3  (-1, 0)          33               1  3.150877
4  (3, -3)          29               0  2.368620


### Gather distribution statistics

Now gather statistics on the distance distribution and return the values. Because each city will take quite a while to run (due to make a lot of Foursquare calls) we don't want to have to rely on the Jupyter kernel to be alive. Next to outputting the numerical values, the function has an option to output the exact string that would be needed to append the results to a list. Once we've run all cities, we'll just create a new notebook and copy-and-paste those strings into the final list. This all can be split across multiple notebooks, which makes them a bit more manageable in size.

The following statistics will be returned:

* Total number of venues obtained overall,
* total number of indicator venues overall,
* the mean (average) distance per venue to the coordinate center,
* standard deviation thereof,
* percentiles (25th, median, 75th, max, interquartile range),
* kurtosis (how much the distribution resembles Gaussian),
* skewedness (left- vs right-tail of the distribution).

Because venues are grouped into coordinate points, there are obvious errors (the venue location isn't exact, and venues may be double-counted). This makes the statistics here dependent on the choice of tiling:

* Shifting (translating) the hex tiling in the plane may vary the statistics because venues may or may not be double counted, and venue distances may flip into another coordinate discretely,
* rotating the hex tiling may have the same effects as shifting (translating).

We argue that the granular choice of the venue tiling is satisfactory for the analysis here. Ideally, we should re-run the analysis with different tiling angles and translations, to quantify these effects and exclude their relevance.

### >>> gather_distribution_stats

In [130]:
def gather_distance_distribution_stats(venues_agg_df, city_index=-1, city_display='(city name, state)', return_as_list_append_string=False):
    
    center_x, center_y, total_num_venues, total_num_indicators = find_venues_distance_to_center(venues_agg_df)
    num_venues_per_coord = venues_agg_df['num_venues']
    dist_per_coord = venues_agg_df['dist']
    
    # blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
    dist_list = []
    for this_coord in zip(num_venues_per_coord, dist_per_coord):
        this_list = [float(this_coord[1])] * int(this_coord[0])
        dist_list.extend(this_list)
    
    dist_list_df = pd.DataFrame(dist_list)
    dist = dist_list_df[0]
    
    # now perform statistics on that new list
    
    mean = dist.mean()
    std = dist.std()
    p25 = dist.quantile(0.25)
    median = dist.quantile(0.5)
    p75 = dist.quantile(0.75)
    max_d = dist.max()
    iqr = p75 - p25
    kurt = dist.kurtosis() # "peaky-ness"
    mad = dist.mad() # mean absolute deviation
    skew = dist.skew() # skewedness
    
    if return_as_list_append_string:
        retS = '# Create list at the beginning:\n'
        retS += '#     l = []\n\n'
        retS += 'l.append(['
        retS += '{}, "{}", ({:.2f}, {:.2f}), '.format(city_index, city_display, center_x, center_y)
        retS += '{}, {}, {}, {}, '.format(total_num_venues, total_num_indicators, mean, std)
        retS += '{}, {}, {}, {}, '.format(p25, median, p75, max_d)
        retS += '{}, {}, {}, {}'.format(iqr, kurt, mad, skew)
        retS += '])\n\n'
        retS += '# Convert list to pandas DataFrame:\n'
        retS += '#     cities_stats_df = pd.DataFrame(l)\n'
        retS += '#     cities_stats_df.columns = ['
        retS += '"city index", "city name, state", "center coord", "total num venues", "total num indicators", '
        retS += '"mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", '
        retS += '"interquartile range", "kurtosis", "mean absolute deviation", "skewedness"'
        retS += ']\n'
        return retS

    else:
        return center_x, center_y, total_num_venues, total_num_indicators, \
               mean, std, p25, median, p75, max_d,                         \
               iqr, kurt, mad, skew
    


Testing this function on the above sample set from Wichita:

In [131]:
center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew \
    = gather_distance_distribution_stats(venues_agg_df)

print(center_x, center_y, total_num_venues, total_num_indicators, \
           mean, std, p25, median, p75, max_d,                         \
           iqr, kurt, mad, skew)

list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=2,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nlist_append_string:\n\n{}'.format(list_append_string))


2.03808752026 -0.835494327391 1234 24 1.730578350699109 0.7969461547276987 1.1651283709269875 1.8358894533937968 2.2026918784384373 3.5495091501 1.0375635075114498 -0.496449614411 0.667242283893936 0.0981070727896

list_append_string:

# Create list at the beginning:
#     l = []

l.append([2, "Wichita, Kansas", (2.04, -0.84), 1234, 24, 1.730578350699109, 0.7969461547276987, 1.1651283709269875, 1.8358894533937968, 2.2026918784384373, 3.5495091500992766, 1.0375635075114498, -0.4964496144109849, 0.667242283893936, 0.09810707278956722])

# Convert list to pandas DataFrame:
#     cities_stats_df = pd.DataFrame(l)
#     cities_stats_df.columns = ["city index", "city name, state", "center coord", "total num venues", "total num indicators", "mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", "interquartile range", "kurtosis", "mean absolute deviation", "skewedness"]



Let's see whether the copy-and-paste magic works:

In [133]:
# copy-and-paste from the function output:

l = []

l.append([2, "Wichita, Kansas", (2.04, -0.84), 1234, 24, 1.730578350699109, 0.7969461547276987, 1.1651283709269875, 1.8358894533937968, 2.2026918784384373, 3.5495091500992766, 1.0375635075114498, -0.4964496144109849, 0.667242283893936, 0.09810707278956722])

cities_stats_df = pd.DataFrame(l)
cities_stats_df.columns = ["city index", "city name, state", "center coord", "total num venues", "total num indicators", "mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", "interquartile range", "kurtosis", "mean absolute deviation", "skewedness"]

# let's have a look:
cities_stats_df


,city index,"city name, state",center coord,total num venues,total num indicators,mean distance to center,std dev,25th percentile,median,75th percentile,max distance to center,interquartile range,kurtosis,mean absolute deviation,skewedness
0,2,"Wichita, Kansas","(2.04, -0.84)",1234,24,1.730578,0.796946,1.165128,1.835889,2.202692,3.549509,1.037564,-0.49645,0.667242,0.098107


Works!

## Put it to action: CITY_INDEX=0

Closing out this notebook, let's see what the values are for the first city.

In [135]:
CITY_INDEX=0
RADIUS=800
MAX_COORDS=1000
MAP_ZOOM_LEVEL=11
MARKER_ZOOM=0.1


In [136]:
lat_orig, long_orig, venues_dict, city_name, city_state = find_venues_geo_distribution(
    cities_df, city_index=CITY_INDEX, radius=RADIUS, max_coords_tested=MAX_COORDS, verbose=False)


[test #1 of 1000] (0,0):22. [test #2 of 1000] (-1,0):11. (1,0):24. (0,-1):21. (0,1):39. (-1,1):22. (1,-1):21. [test #3 of 1000] (1,1):39. (0,2):34. (-1,2):24. [test #4 of 1000] (2,1):23. (1,2):35. (2,0):16. [test #5 of 1000] (2,2):18. (1,3):26. (0,3):25. [test #6 of 1000] (-1,3):18. [test #7 of 1000] (2,3):5. (1,4):13. (0,4):38. [test #8 of 1000] (-1,4):23. (0,5):29. (-1,5):33. [test #9 of 1000] (-2,5):29. (-1,6):38. (-2,6):31. [test #10 of 1000] (0,6):19. (-1,7):37. (-2,7):31. [test #11 of 1000] (0,7):41. (-1,8):52. (-2,8):39. [test #12 of 1000] (0,8):35. (-1,9):34. (-2,9):33. [test #13 of 1000] (1,7):27. (1,6):11. [test #14 of 1000] (-3,8):18. (-3,9):19. [test #15 of 1000] (1,8):26. (0,9):33. [test #16 of 1000] (-1,10):26. (-2,10):21. [test #17 of 1000] (-3,10):20. [test #18 of 1000] (1,9):17. (0,10):39. [test #19 of 1000] (1,10):22. (0,11):39. (-1,11):51. [test #20 of 1000] (-2,11):32. (-1,12):97. (-2,12):68. [test #21 of 1000] (0,12):58. (-1,13):62. (-2,13):77. [test #22 of 1000] (

44. (17,-7):28. [test #211 of 1000] (17,-5):43. (16,-4):12. [test #212 of 1000] (14,-3):2. [test #213 of 1000] (18,-6):28. (18,-7):34. [test #214 of 1000] (13,-6):4. (14,-7):3. [test #215 of 1000] (15,-3):1. [test #216 of 1000] (18,-5):14. (17,-4):5. [test #217 of 1000] (19,-7):17. (18,-8):7. (19,-8):5. [test #218 of 1000] (12,-5):4. [test #219 of 1000] (17,-8):5. [test #220 of 1000] (19,-6):12. [test #221 of 1000] [test #222 of 1000] [test #223 of 1000] (-4,8):6. [test #224 of 1000] (6,13):7. [test #225 of 1000] (3,8):8. (3,7):6. [test #226 of 1000] (9,-1):4. [test #227 of 1000] (5,-8):13. (6,-8):12. [test #228 of 1000] [test #229 of 1000] (-7,13):12. (-6,12):16. [test #230 of 1000] (-9,10):4. (-10,10):5. [test #231 of 1000] [test #232 of 1000] [test #233 of 1000] [test #234 of 1000] (0,17):5. (1,18):29. (0,18):4. [test #235 of 1000] (2,18):32. (1,19):29. (0,19):27. [test #236 of 1000] (3,18):5. (2,19):25. [test #237 of 1000] (1,20):32. (0,20):29. [test #238 of 1000] (2,20):15. (1,21)

7. [test #446 of 1000] (9,-27):4. (10,-27):14. [test #447 of 1000] (11,-25):12. (11,-26):26. [test #448 of 1000] [test #449 of 1000] (11,-27):25. [test #450 of 1000] (12,-26):27. (12,-27):43. [test #451 of 1000] (13,-27):26. (12,-28):32. (13,-28):38. [test #452 of 1000] (14,-28):25. (13,-29):24. (14,-29):22. [test #453 of 1000] (11,-28):9. (12,-29):13. [test #454 of 1000] (13,-26):14. (12,-25):5. [test #455 of 1000] (14,-27):17. [test #456 of 1000] [test #457 of 1000] (15,-28):6. (15,-29):11. [test #458 of 1000] (13,-30):22. (14,-30):17. [test #459 of 1000] (6,-25):4. (7,-26):5. [test #460 of 1000] (15,-30):14. [test #461 of 1000] (12,-30):15. (13,-31):19. (14,-31):11. [test #462 of 1000] (12,-31):11. (13,-32):11. (14,-32):21. [test #463 of 1000] (15,-32):10. (14,-33):9. (15,-33):3. [test #464 of 1000] (9,-22):14. [test #465 of 1000] (10,-23):7. [test #466 of 1000] (15,-27):4. (14,-26):4. [test #467 of 1000] (15,-31):11. [test #468 of 1000] (8,-27):4. [test #469 of 1000] (11,-30):7. (1

2. [test #665 of 1000] [test #666 of 1000] (24,-27):0. (23,-26):1. [test #667 of 1000] (24,-31):2. [test #668 of 1000] (20,-32):1. (21,-32):1. [test #669 of 1000] (22,-32):2. [test #670 of 1000] (13,-34):2. [test #671 of 1000] (-1,-12):5. (0,-13):4. (-1,-11):7. [test #672 of 1000] [test #673 of 1000] (15,-14):5. [test #674 of 1000] (13,-13):1. (14,-14):5. [test #675 of 1000] (10,-11):2. (11,-12):9. [test #676 of 1000] (12,-13):4. [test #677 of 1000] [test #678 of 1000] (19,5):5. (18,5):8. [test #679 of 1000] (17,5):5. (18,6):6. (17,6):5. [test #680 of 1000] (14,4):1. [test #681 of 1000] [test #682 of 1000] [test #683 of 1000] (21,-1):4. [test #684 of 1000] (-10,22):4. (-9,23):1. (-10,23):0. [test #685 of 1000] (18,-23):5. (17,-22):3. (18,-24):5. [test #686 of 1000] (19,-26):2. [test #687 of 1000] (21,-25):2. (20,-25):2. [test #688 of 1000] (-2,-11):13. (-1,-10):9. (-2,-10):14. [test #689 of 1000] (-3,-10):15. (-2,-9):11. (-3,-9):14. [test #690 of 1000] (-4,-10):7. (-3,-11):5. (-4,-9):9

47. (-15,15):15. [test #908 of 1000] (-15,13):35. (-14,12):34. (-13,12):37. [test #909 of 1000] (-12,12):7. [test #910 of 1000] (-16,14):12. (-16,15):8. [test #911 of 1000] (-13,11):11. (-12,11):6. [test #912 of 1000] (-16,13):5. (-15,12):10. [test #913 of 1000] (-14,11):9. [test #914 of 1000] (-15,16):3. (-16,16):5. [test #915 of 1000] (-17,14):2. (-17,15):1. [test #916 of 1000] [test #917 of 1000] (-14,16):1. [test #918 of 1000] (-13,10):2. [test #919 of 1000] (-16,12):4. (-15,11):3. [test #920 of 1000] [test #921 of 1000] (-14,10):1. [test #922 of 1000] (-17,16):0. [test #923 of 1000] [test #924 of 1000] (-13,16):2. [test #925 of 1000] [test #926 of 1000] [test #927 of 1000] [test #928 of 1000] [test #929 of 1000] (18,-25):1. (18,-26):3. [test #930 of 1000] (17,-27):3. [test #931 of 1000] [test #932 of 1000] (6,-26):4. (7,-27):3. [test #933 of 1000] (9,-28):4. [test #934 of 1000] [test #935 of 1000] (11,-33):2. (12,-34):2. [test #936 of 1000] [test #937 of 1000] [test #938 of 1000] 

In [211]:
make_map_from_dict(
    lat_orig, long_orig, venues_dict, 11, city_name, city_state, radius_zoom=7,
    width=600, height=600, simple_rendering=True)


In [171]:
print("{}, {}".format(city_name, city_state))
venues_agg_df, venues_types = aggregate_venues_dict(venues_dict)
print('Total shape: {}\nSample 10 venues per coordinate (unsorted):'.format(venues_agg_df.shape))
venues_agg_df.head(10)


Arlington, Texas
Total shape: (1429, 3)
Sample 10 venues per coordinate (unsorted):


,coord,num_venues,num_indicators
0,"(-10, -8)",21,1
1,"(18, -29)",5,0
2,"(-7, -3)",4,0
3,"(22, -5)",48,1
4,"(16, -31)",12,1
5,"(-7, 2)",17,0
6,"(-7, 15)",17,1
7,"(17, -6)",44,0
8,"(9, 0)",11,0
9,"(-5, 1)",28,1


In [141]:
list_append_string = gather_distance_distribution_stats(
    venues_agg_df,
    city_index=CITY_INDEX,
    city_display='{}, {}'.format(city_name, city_state),
    return_as_list_append_string=True)

print('\nResulting list append string for this city:\n\n{}'.format(list_append_string))



Resulting list append string for this city:

# Create list at the beginning:
#     l = []

l.append([0, "Arlington, Texas", (4.84, -1.87), 19691, 537, 14.338724874214476, 7.08435831078514, 9.023802690403183, 14.214585770360175, 18.489538701142298, 34.86736270444082, 9.465736010739114, -0.28589232273629905, 5.652171632302203, 0.37546709584470855])

# Convert list to pandas DataFrame:
#     cities_stats_df = pd.DataFrame(l)
#     cities_stats_df.columns = ["city index", "city name, state", "center coord", "total num venues", "total num indicators", "mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", "interquartile range", "kurtosis", "mean absolute deviation", "skewedness"]



In [142]:
# testing the copy-and-paste functions one more time:
l = []
l.append([0, "Arlington, Texas", (4.84, -1.87), 19691, 537, 14.338724874214476, 7.08435831078514, 9.023802690403183, 14.214585770360175, 18.489538701142298, 34.86736270444082, 9.465736010739114, -0.28589232273629905, 5.652171632302203, 0.37546709584470855])
cities_stats_df = pd.DataFrame(l)
cities_stats_df.columns = ["city index", "city name, state", "center coord", "total num venues", "total num indicators", "mean distance to center", "std dev", "25th percentile", "median", "75th percentile", "max distance to center", "interquartile range", "kurtosis", "mean absolute deviation", "skewedness"]

# show it
cities_stats_df



,city index,"city name, state",center coord,total num venues,total num indicators,mean distance to center,std dev,25th percentile,median,75th percentile,max distance to center,interquartile range,kurtosis,mean absolute deviation,skewedness
0,0,"Arlington, Texas","(4.84, -1.87)",19691,537,14.338725,7.084358,9.023803,14.214586,18.489539,34.867363,9.465736,-0.285892,5.652172,0.375467


## Validation using Pandas profiling

In order to get an idea that these aggregates are OK, use Pandas profiling on the list of venue distances in raw form

In [143]:
num_venues_per_coord = venues_agg_df['num_venues']
dist_per_coord = venues_agg_df['dist']
    
# blow up the dist_per_coord series by the nunmber of venues at the coordinate
    
dist_list = []
for this_coord in zip(num_venues_per_coord, dist_per_coord):
    this_list = [float(this_coord[1])] * int(this_coord[0])
    dist_list.extend(this_list)
    
dist_list_df = pd.DataFrame(dist_list)

pandas_profiling.ProfileReport(dist_list_df)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/matplotlib/font_manager.py:1316: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/matplotlib/font_manager.py:1316: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans
  (prop.get_family(), self.defaultFamily[fontext]))


Number of variables,1
Number of observations,19691
Total Missing (%),0.0%
Total size in memory,153.9 KiB
Average record size in memory,8.0 B
Numeric,1
Categorical,0
Boolean,0
Date,0
Text (Unique),0
Rejected,0


The analysis values match the dataframe output above. By eyeballing the map, they also look plausible.

We're ready to run this for all cities.